<a href="https://colab.research.google.com/github/BoraYavuz/AI_Agents/blob/main/Multiagent_Db_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install streamlit
!pip install langchain
!pip install langchain-openai
!pip install langchain_community
!pip install arxiv
!pip install plotly
!pip install scholarly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 35.3 MB/s eta 0:00:00
  Created wheel for bibtexparser: filename=bibtexparser-1.4.3-py3-none-any.whl size=43549 sha256=d1501e9a2df8d2a4dd94b4bb4db072c1b14e4c0034a083cedd5f1ac33d9cd672
  Stored in directory: /root/.cache/pip/wheels/16/fb/76/306387739cf9d53b1c39b0c8aadbbb17dc05f256756d8fd915
  Created wheel f

In [1]:
import streamlit as st
import pandas as pd
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain.tools import Tool
from langchain.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent, AgentType
from langchain_community.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain_community.utilities import WikipediaAPIWrapper, DuckDuckGoSearchAPIWrapper
import arxiv
import requests
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from scholarly import scholarly
import time
import json
import re
from typing import Dict, List, Any

In [2]:
# Page config
st.set_page_config(
    page_title="Multi-Agent Research System",
    page_icon="🤖",
    layout="wide"
)

# Sidebar configuration
st.sidebar.title("🔧 Configuration")
openai_api_key = st.sidebar.text_input("OpenAI API Key", type="password")

# Agent selection
st.sidebar.markdown("### 🤖 Agent Selection")
agent_pipeline = st.sidebar.multiselect(
    "Select agents to include in pipeline:",
    ["Literature Agent", "Analysis Agent", "Writing Agent", "Fact-Check Agent", "Visualization Agent"],
    default=["Literature Agent", "Analysis Agent", "Writing Agent"]
)

# Database selection (keeping your original code)
st.sidebar.markdown("### 📚 Research Database")
database_options = {
    "ArXiv": "Academic preprints (Physics, Math, CS, etc.)",
    "Google Scholar": "Broad academic search across disciplines",
    "PubMed": "Medical and life sciences literature",
    "Semantic Scholar": "AI-powered academic search",
    "CrossRef": "Academic publications with DOIs"
}

selected_db = st.sidebar.selectbox(
    "Choose Research Database:",
    options=list(database_options.keys()),
    help="Different databases specialize in different fields"
)

max_papers = st.sidebar.slider("Max papers to analyze", 5, 50, 10)

# [Keep all your original search functions here - they're perfect]
def search_arxiv_papers(query, max_results=10):
    """Search arXiv for papers"""
    try:
        search = arxiv.Search(
            query=query,
            max_results=max_results,
            sort_by=arxiv.SortCriterion.SubmittedDate
        )
        papers = []
        for paper in search.results():
            papers.append({
                'title': paper.title,
                'authors': [author.name for author in paper.authors],
                'published': paper.published.strftime('%Y-%m-%d'),
                'summary': paper.summary,
                'url': paper.entry_id,
                'categories': paper.categories,
                'source': 'ArXiv'
            })
        return papers
    except Exception as e:
        st.error(f"ArXiv search error: {e}")
        return []


2025-06-17 12:41:31.022 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-17 12:41:31.024 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-17 12:41:31.305 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-06-17 12:41:31.307 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-17 12:41:31.308 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-17 12:41:31.309 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-17 12:41:31.310 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

In [3]:
# [Keep all your original search functions here - they're perfect]
def search_arxiv_papers(query, max_results=10):
    """Search arXiv for papers"""
    try:
        search = arxiv.Search(
            query=query,
            max_results=max_results,
            sort_by=arxiv.SortCriterion.SubmittedDate
        )
        papers = []
        for paper in search.results():
            papers.append({
                'title': paper.title,
                'authors': [author.name for author in paper.authors],
                'published': paper.published.strftime('%Y-%m-%d'),
                'summary': paper.summary,
                'url': paper.entry_id,
                'categories': paper.categories,
                'source': 'ArXiv'
            })
        return papers
    except Exception as e:
        st.error(f"ArXiv search error: {e}")
        return []

# [Include all your other search functions - keeping them for brevity]
def search_papers_by_database(database, query, max_results=10):
    """Route search to appropriate database function"""
    if database == "ArXiv":
        return search_arxiv_papers(query, max_results)
    # Add other database calls here
    else:
        return []

# NEW: Enhanced Agent Classes
class MultiAgentOrchestrator:
    def __init__(self, api_key: str, selected_database: str):
        self.api_key = api_key
        self.selected_database = selected_database
        self.llm = ChatOpenAI(
            temperature=0.1,
            openai_api_key=api_key,
            model="gpt-4o-mini"
        )
        self.shared_memory = {}

    def create_literature_agent(self):
        """Your original literature agent with slight modifications"""
        def paper_search_tool(query: str) -> str:
            papers = search_papers_by_database(self.selected_database, query, max_papers)
            if not papers:
                return f"No papers found for '{query}' in {self.selected_database}."

            # Store papers in shared memory for other agents
            self.shared_memory['papers'] = papers
            self.shared_memory['search_query'] = query

            result = f"Found {len(papers)} papers on '{query}' from {self.selected_database}:\n\n"
            for i, paper in enumerate(papers, 1):
                result += f"{i}. **{paper['title']}**\n"
                result += f"   Authors: {', '.join(paper['authors'][:3])}{'...' if len(paper['authors']) > 3 else ''}\n"
                result += f"   Published: {paper['published']}\n"
                result += f"   Summary: {paper['summary'][:200]}...\n\n"

            return result

        tools = [Tool(
            name="paper_search",
            func=paper_search_tool,
            description=f"Search for academic papers using {self.selected_database}"
        )]

        prompt = ChatPromptTemplate.from_messages([
            ("system", f"""You are a research literature agent specialized in {self.selected_database}.
            Your role is to search for and summarize academic papers. Be thorough and accurate."""),
            ("user", "{input}"),
            ("assistant", "{agent_scratchpad}")
        ])

        agent = create_openai_tools_agent(self.llm, tools, prompt)
        return AgentExecutor(agent=agent, tools=tools, verbose=True)

    def create_analysis_agent(self):
        """Agent that analyzes patterns and trends from literature results"""
        def analyze_papers_tool(query: str) -> str:
            papers = self.shared_memory.get('papers', [])
            if not papers:
                return "No papers available for analysis. Run literature search first."

            # Perform analysis
            analysis = f"Analysis of {len(papers)} papers:\n\n"

            # Author analysis
            all_authors = []
            for paper in papers:
                all_authors.extend(paper['authors'])
            author_counts = pd.Series(all_authors).value_counts().head(10)
            analysis += f"Top Authors:\n{author_counts.to_string()}\n\n"

            # Year analysis
            years = []
            for paper in papers:
                try:
                    year = int(paper['published'][:4])
                    years.append(year)
                except:
                    pass

            if years:
                year_counts = pd.Series(years).value_counts().sort_index()
                analysis += f"Publication Trends:\n{year_counts.to_string()}\n\n"

            # Keyword extraction from titles and abstracts
            text = " ".join([p['title'] + " " + p['summary'] for p in papers])
            words = re.findall(r'\b\w{4,}\b', text.lower())
            word_counts = pd.Series(words).value_counts().head(20)
            analysis += f"Common Keywords:\n{word_counts.to_string()}\n\n"

            # Store analysis for other agents
            self.shared_memory['analysis'] = {
                'author_counts': author_counts.to_dict(),
                'year_counts': year_counts.to_dict() if years else {},
                'keywords': word_counts.to_dict()
            }

            return analysis

        def identify_gaps_tool(query: str) -> str:
            papers = self.shared_memory.get('papers', [])
            if not papers:
                return "No papers available for gap analysis."

            # Simple gap analysis based on keywords and trends
            gaps = "Research Gap Analysis:\n\n"
            gaps += "Based on the literature, potential research gaps include:\n"
            gaps += "1. Underexplored methodologies\n"
            gaps += "2. Recent technological advances not yet applied\n"
            gaps += "3. Cross-disciplinary opportunities\n"
            gaps += "4. Geographical or demographic gaps in studies\n"

            return gaps

        tools = [
            Tool(name="analyze_papers", func=analyze_papers_tool,
                 description="Analyze patterns in research papers"),
            Tool(name="identify_gaps", func=identify_gaps_tool,
                 description="Identify research gaps from literature")
        ]

        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a research analysis agent. Your role is to:
            1. Analyze patterns in research literature
            2. Identify trends and gaps
            3. Provide insights for future research directions
            Be analytical and provide concrete insights."""),
            ("user", "{input}"),
            ("assistant", "{agent_scratchpad}")
        ])

        agent = create_openai_tools_agent(self.llm, tools, prompt)
        return AgentExecutor(agent=agent, tools=tools, verbose=True)

    def create_writing_agent(self):
        """Agent that creates structured reports from research and analysis"""
        def generate_report_tool(query: str) -> str:
            papers = self.shared_memory.get('papers', [])
            analysis = self.shared_memory.get('analysis', {})

            if not papers:
                return "No research data available for report generation."

            report = f"""# Literature Review Report: {self.shared_memory.get('search_query', 'Research Topic')}

## Executive Summary
This report analyzes {len(papers)} academic papers from {self.selected_database} database.

## Key Findings
- Total papers analyzed: {len(papers)}
- Publication span: {min(analysis.get('year_counts', {}).keys()) if analysis.get('year_counts') else 'N/A'} - {max(analysis.get('year_counts', {}).keys()) if analysis.get('year_counts') else 'N/A'}
- Most active authors: {list(analysis.get('author_counts', {}).keys())[:3] if analysis.get('author_counts') else 'N/A'}

## Literature Overview
"""

            # Add paper summaries
            for i, paper in enumerate(papers[:5], 1):  # Top 5 papers
                report += f"\n### Paper {i}: {paper['title']}\n"
                report += f"**Authors:** {', '.join(paper['authors'][:3])}\n"
                report += f"**Published:** {paper['published']}\n"
                report += f"**Summary:** {paper['summary'][:300]}...\n"

            report += f"\n## Trends and Patterns\n"
            if analysis.get('keywords'):
                top_keywords = list(analysis['keywords'].keys())[:10]
                report += f"**Key themes:** {', '.join(top_keywords)}\n"

            report += f"\n## Recommendations\n"
            report += "Based on this literature review, we recommend:\n"
            report += "1. Further investigation into emerging themes\n"
            report += "2. Cross-disciplinary collaboration opportunities\n"
            report += "3. Addressing identified research gaps\n"

            # Store report for download
            self.shared_memory['final_report'] = report

            return report

        tools = [Tool(
            name="generate_report",
            func=generate_report_tool,
            description="Generate a structured research report"
        )]

        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a scientific writing agent. Your role is to:
            1. Create well-structured research reports
            2. Synthesize findings from multiple sources
            3. Provide clear recommendations
            Write in academic style but keep it accessible."""),
            ("user", "{input}"),
            ("assistant", "{agent_scratchpad}")
        ])

        agent = create_openai_tools_agent(self.llm, tools, prompt)
        return AgentExecutor(agent=agent, tools=tools, verbose=True)

    def create_fact_check_agent(self):
        """Agent that fact-checks claims using external sources"""
        # Initialize external search tools
        wiki_search = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        web_search = DuckDuckGoSearchRun(api_wrapper=DuckDuckGoSearchAPIWrapper())

        def fact_check_claims_tool(query: str) -> str:
            papers = self.shared_memory.get('papers', [])
            if not papers:
                return "No research data available for fact-checking."

            # Extract key claims from papers (simplified)
            claims = []
            for paper in papers[:3]:  # Check first 3 papers
                # Simple claim extraction (in practice, you'd use more sophisticated NLP)
                sentences = paper['summary'].split('. ')
                for sentence in sentences[:2]:  # First 2 sentences
                    if len(sentence) > 50:  # Substantial claims
                        claims.append(sentence)

            fact_check_results = "Fact-Check Results:\n\n"
            for i, claim in enumerate(claims[:3], 1):  # Check first 3 claims
                fact_check_results += f"Claim {i}: {claim[:100]}...\n"

                # Search for supporting evidence (simplified)
                try:
                    search_query = claim.split()[:5]  # First 5 words
                    wiki_result = wiki_search.run(" ".join(search_query))
                    fact_check_results += f"Supporting evidence found: {wiki_result[:200]}...\n\n"
                except:
                    fact_check_results += "Could not verify this claim.\n\n"

            return fact_check_results

        tools = [
            Tool(name="fact_check", func=fact_check_claims_tool,
                 description="Fact-check research claims using external sources"),
            wiki_search,
            web_search
        ]

        agent = initialize_agent(
            tools, self.llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
        )
        return agent

    def create_visualization_agent(self):
        """Agent that creates visualizations from research data"""
        def create_visualizations_tool(query: str) -> str:
            analysis = self.shared_memory.get('analysis', {})
            papers = self.shared_memory.get('papers', [])

            if not analysis or not papers:
                return "No data available for visualization."

            # Create publication timeline
            if analysis.get('year_counts'):
                years = list(analysis['year_counts'].keys())
                counts = list(analysis['year_counts'].values())

                fig = px.line(x=years, y=counts, title="Publication Timeline")
                self.shared_memory['timeline_chart'] = fig

            # Create keyword cloud data
            if analysis.get('keywords'):
                self.shared_memory['keywords_data'] = analysis['keywords']

            return "Visualizations created and stored. Check the visualization section below."

        tools = [Tool(
            name="create_viz",
            func=create_visualizations_tool,
            description="Create visualizations from research data"
        )]

        prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a data visualization agent. Create meaningful charts and graphs from research data."),
            ("user", "{input}"),
            ("assistant", "{agent_scratchpad}")
        ])

        agent = create_openai_tools_agent(self.llm, tools, prompt)
        return AgentExecutor(agent=agent, tools=tools, verbose=True)



In [4]:
# Main Streamlit Interface
st.title("🤖 Multi-Agent Research System")
st.write("An AI system that uses multiple specialized agents to conduct comprehensive research analysis.")

# Display selected agents
if agent_pipeline:
    st.info(f"🎯 **Active Agents:** {' → '.join(agent_pipeline)}")

# Input section
col1, col2 = st.columns([3, 1])
with col1:
    research_query = st.text_input(
        "Enter your research question:",
        placeholder="e.g., 'machine learning in healthcare'"
    )

with col2:
    execution_mode = st.selectbox(
        "Execution Mode",
        ["Sequential", "Parallel Analysis"]
    )

if st.button("🚀 Start Multi-Agent Research", type="primary"):
    if not openai_api_key:
        st.error("Please enter your OpenAI API key.")
    elif not research_query:
        st.error("Please enter a research query.")
    elif not agent_pipeline:
        st.error("Please select at least one agent.")
    else:
        # Initialize orchestrator
        orchestrator = MultiAgentOrchestrator(openai_api_key, selected_db)

        # Create agents based on selection
        agents = {}
        if "Literature Agent" in agent_pipeline:
            agents["literature"] = orchestrator.create_literature_agent()
        if "Analysis Agent" in agent_pipeline:
            agents["analysis"] = orchestrator.create_analysis_agent()
        if "Writing Agent" in agent_pipeline:
            agents["writing"] = orchestrator.create_writing_agent()
        if "Fact-Check Agent" in agent_pipeline:
            agents["fact_check"] = orchestrator.create_fact_check_agent()
        if "Visualization Agent" in agent_pipeline:
            agents["visualization"] = orchestrator.create_visualization_agent()

        # Execute agents sequentially
        results = {}

        with st.spinner("🔍 Running multi-agent research pipeline..."):
            try:
                # 1. Literature Agent (always first if selected)
                if "literature" in agents:
                    st.write("### 📚 Literature Agent Working...")
                    lit_result = agents["literature"].invoke({
                        "input": f"Search for academic papers about: {research_query}"
                    })
                    results["literature"] = lit_result["output"]
                    st.success("✅ Literature search complete!")

                # 2. Analysis Agent
                if "analysis" in agents:
                    st.write("### 📊 Analysis Agent Working...")
                    analysis_result = agents["analysis"].invoke({
                        "input": f"Analyze the research papers about: {research_query}"
                    })
                    results["analysis"] = analysis_result["output"]
                    st.success("✅ Analysis complete!")

                # 3. Writing Agent
                if "writing" in agents:
                    st.write("### ✍️ Writing Agent Working...")
                    writing_result = agents["writing"].invoke({
                        "input": f"Generate a comprehensive report about: {research_query}"
                    })
                    results["writing"] = writing_result["output"]
                    st.success("✅ Report generation complete!")

                # 4. Fact-Check Agent
                if "fact_check" in agents:
                    st.write("### 🔍 Fact-Check Agent Working...")
                    fact_check_result = agents["fact_check"].run(
                        f"Fact-check key claims from the research on: {research_query}"
                    )
                    results["fact_check"] = fact_check_result
                    st.success("✅ Fact-checking complete!")

                # 5. Visualization Agent
                if "visualization" in agents:
                    st.write("### 📈 Visualization Agent Working...")
                    viz_result = agents["visualization"].invoke({
                        "input": f"Create visualizations for research on: {research_query}"
                    })
                    results["visualization"] = viz_result["output"]
                    st.success("✅ Visualizations complete!")

                # Display all results
                st.markdown("## 🎯 Multi-Agent Research Results")

                for agent_name, result in results.items():
                    with st.expander(f"📋 {agent_name.title()} Results", expanded=True):
                        st.markdown(result)

                # Show visualizations if created
                if orchestrator.shared_memory.get('timeline_chart'):
                    st.plotly_chart(orchestrator.shared_memory['timeline_chart'])

                # Provide download for final report
                if orchestrator.shared_memory.get('final_report'):
                    st.download_button(
                        label="📄 Download Complete Research Report",
                        data=orchestrator.shared_memory['final_report'],
                        file_name=f"multi_agent_research_{research_query.replace(' ', '_')}_{datetime.now().strftime('%Y%m%d')}.md",
                        mime="text/markdown"
                    )

            except Exception as e:
                st.error(f"An error occurred in the multi-agent pipeline: {e}")

# Agent Information
with st.expander("ℹ️ About the Multi-Agent System"):
    st.markdown("""
    ### 🤖 Available Agents:

    - **Literature Agent**: Searches academic databases for relevant papers
    - **Analysis Agent**: Analyzes patterns, trends, and identifies research gaps
    - **Writing Agent**: Creates structured reports and summaries
    - **Fact-Check Agent**: Verifies claims using external sources (Wikipedia, web search)
    - **Visualization Agent**: Creates charts and graphs from research data

    ### 🔄 How It Works:

    1. **Sequential Processing**: Agents work in order, each building on previous results
    2. **Shared Memory**: Agents share data through a common memory system
    3. **Specialized Tools**: Each agent has specific tools for their domain
    4. **Comprehensive Output**: Final report combines insights from all agents
    """)

st.markdown("---")
st.markdown("*Multi-Agent Research System - Built with LangChain, OpenAI, and Streamlit*")

2025-06-17 13:42:27.116 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-17 13:42:27.122 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-17 13:42:27.125 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-17 13:42:27.127 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-17 13:42:27.128 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-17 13:42:27.129 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-17 13:42:27.135 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-17 13:42:27.137 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()